In [175]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import os

DIR = '../midi_results/pianomidi_gridsearch_fair/'

In [176]:


experiments = os.listdir(DIR)

gate_expmts = [exp for exp in experiments if 'gate' in exp and 'comb' not in exp and 'linear' not in exp]
combo_expmts = [exp for exp in experiments if 'comb' in exp and 'linear' not in exp and 'gate' in exp]
gatelin_expmts = [exp for exp in experiments if 'gate' in exp and 'comb' not in exp and 'linear' in exp]
combolin_expmts = [exp for exp in experiments if 'comb' in exp and 'linear' in exp]
bilin_expmts = [exp for exp in experiments if 'simple' in exp and 'comb' not in exp]
combobilin_expmts = [exp for exp in experiments if 'simple' in exp and 'comb' in exp]
gru_expmts = [exp for exp in experiments if 'gru' in exp]
lstm_expmts = [exp for exp in experiments if 'lstm' in exp]
van_expts = [exp for exp in experiments if 'vanilla' in exp]
print(combo_expmts)
print(bilin_expmts)

['cp-gate-combined-0.001-8-100-rankdouble', 'cp-gate-combined-0.001-8-100-rankfull', 'cp-gate-combined-0.001-8-100-rankhalf', 'cp-gate-combined-0.001-8-100-rankone', 'cp-gate-combined-0.001-8-75-rankdouble', 'cp-gate-combined-0.001-8-75-rankfull', 'cp-gate-combined-0.001-8-75-rankhalf', 'cp-gate-combined-0.001-8-75-rankone', 'cp-gate-combined-0.01-8-100-rankdouble', 'cp-gate-combined-0.01-8-100-rankfull', 'cp-gate-combined-0.01-8-100-rankhalf', 'cp-gate-combined-0.01-8-100-rankone', 'cp-gate-combined-0.01-8-75-rankdouble', 'cp-gate-combined-0.01-8-75-rankfull', 'cp-gate-combined-0.01-8-75-rankhalf', 'cp-gate-combined-0.01-8-75-rankone', 'cp-gate-combined-0.1-8-100-rankdouble', 'cp-gate-combined-0.1-8-100-rankfull', 'cp-gate-combined-0.1-8-100-rankhalf', 'cp-gate-combined-0.1-8-100-rankone', 'cp-gate-combined-0.1-8-75-rankdouble', 'cp-gate-combined-0.1-8-75-rankfull', 'cp-gate-combined-0.1-8-75-rankhalf', 'cp-gate-combined-0.1-8-75-rankone']
['simple_cp-0.001-8-100-rankdouble', 'simple_

In [177]:
def get_nll(result):
    return float(result.split(':')[-1])

def get_xent(result):
    # more awkward
    return float(result.split(':')[1].split(',')[0])

def best_expts(all_expts, directory=DIR):
    best_val_expt = ''
    best_test_expt = ''
    best_val_err = 100
    best_test_err = 100
    
    for exp in all_expts:
        results_path = os.path.join(directory, exp, 'earlystopped_results.txt')
        # open it up and find the numbers
        if not os.path.exists(results_path):
            print('no results for: {}'.format(results_path))
            continue
        with open(results_path) as fp:
            test, valid, train, _ = fp.read().split('\n')
            # now split out the nll
            test_nll = get_nll(test)
            valid_nll = get_nll(valid)
            
            if test_nll < best_test_err:
                best_test_err = test_nll
                best_test_expt = exp
#             if valid_nll < best_val_err:
#                 best_val_err = valid_nll
#                 best_val_expt = exp
    return best_test_expt, #best_val_expt

In [178]:
def format_results(exp, directory=DIR):
    splits = exp.split('-')
    if len(splits) >= 6:
        num = len(splits) - 4
        cell = '-'.join(splits[:num])
        lr, bs, sl, rank = splits[num:]
    elif len(splits) == 5:
        cell, lr, bs, sl, rank = splits
    
    print('CELL: {}'.format(cell))
    print('~lr: {}, bs: {}, sl: {}, rk: {}~'.format(lr, bs, sl, rank))
    result_path = os.path.join(directory, exp, 'earlystopped_results.txt')
    nlls = []
    with open(result_path) as fp:
        for line in fp:
            print(line, end='')
            nlls.append(get_nll(line))
    print('\t& '.join(['{:.4f}'.format(num) for num in nlls[::-1]]) + '   \\\\')

In [179]:
for expt in best_expts(gate_expmts):
    format_results(expt)

CELL: cp-gate
~lr: 0.001, bs: 8, sl: 100, rk: rankhalf~
Test  xent: 0.0876821473888729, nll: 7.71608481199845
Valid xent: 0.09770531728863716, nll: 8.59814805984497
Train xent: 0.08196723801975554, nll: 7.213172511851534
7.2132	& 8.5981	& 7.7161   \\


In [180]:
for expt in best_expts(combo_expmts):
    format_results(expt)

CELL: cp-gate-combined
~lr: 0.001, bs: 8, sl: 75, rk: rankhalf~
Test  xent: 0.0875034995617405, nll: 7.70032944217805
Valid xent: 0.0970242518399443, nll: 8.53816546712603
Train xent: 0.08266847078052778, nll: 7.274846996579852
7.2748	& 8.5382	& 7.7003   \\


In [181]:
for expt in best_expts(gatelin_expmts):
    format_results(expt)

CELL: cp-gate-linear
~lr: 0.001, bs: 8, sl: 75, rk: rankhalf~
Test  xent: 0.08803927874372851, nll: 7.747472778443368
Valid xent: 0.09750756035958018, nll: 8.580695969717842
Train xent: 0.08280152673759157, nll: 7.286554798247322
7.2866	& 8.5807	& 7.7475   \\


In [182]:
for expt in best_expts(combolin_expmts):
    format_results(expt)

CELL: cp-gate-combined-linear
~lr: 0.001, bs: 8, sl: 75, rk: rankhalf~
Test  xent: 0.08773684044999461, nll: 7.720866864727389
Valid xent: 0.09740439749189786, nll: 8.571622133255005
Train xent: 0.0832002382311556, nll: 7.321642663743761
7.3216	& 8.5716	& 7.7209   \\


In [183]:
for expt in best_expts(lstm_expmts):
    format_results(expt)

CELL: lstm
~lr: 0.001, bs: 8, sl: 75, rk: rankone~
Test  xent: 0.08811495909767766, nll: 7.754166679997598
Valid xent: 0.09696459184799876, nll: 8.532946859087263
Train xent: 0.08378386627587062, nll: 7.37302951964121
7.3730	& 8.5329	& 7.7542   \\


In [184]:
for expt in best_expts(gru_expmts):
    format_results(expt)

CELL: gru
~lr: 0.001, bs: 8, sl: 100, rk: rankone~
Test  xent: 0.08824089104714601, nll: 7.765248360841171
Valid xent: 0.09735247492790222, nll: 8.56708083152771
Train xent: 0.08299829842562371, nll: 7.303899389632205
7.3039	& 8.5671	& 7.7652   \\


In [185]:
for expt in best_expts(van_expts):
    format_results(expt)

CELL: vanilla
~lr: 0.001, bs: 8, sl: 75, rk: rankone~
Test  xent: 0.08874140175119523, nll: 7.80929377771193
Valid xent: 0.09767656134707588, nll: 8.59560411317008
Train xent: 0.08394747493522507, nll: 7.387422599489727
7.3874	& 8.5956	& 7.8093   \\


In [187]:
for expt in best_expts(bilin_expmts):
    format_results(expt)

CELL: simple_cp
~lr: 0.001, bs: 8, sl: 100, rk: rankone~
Test  xent: 0.08826707076767216, nll: 7.767574621283489
Valid xent: 0.0976012259721756, nll: 8.588997268676758
Train xent: 0.0836337574143359, nll: 7.359835365985302
7.3598	& 8.5890	& 7.7676   \\


In [186]:
for expt in best_expts(combobilin_expmts):
    format_results(expt)

no results for: ../midi_results/pianomidi_gridsearch_fair/simple_cp-combined-0.1-8-100-rankone/earlystopped_results.txt
CELL: simple_cp-combined
~lr: 0.001, bs: 8, sl: 100, rk: rankhalf~
Test  xent: 0.08933468940465347, nll: 7.861531630806301
Valid xent: 0.09926063790917397, nll: 8.735036611557007
Train xent: 0.08436769025122866, nll: 7.424429264474423
7.4244	& 8.7350	& 7.8615   \\
